### Install necessary libraries

In [1]:
from IPython.display import clear_output

# !apt-get install libmagickwand-dev
!pip install ipython-autotime
# !apt-get update
# !apt install ghostscript python3-tk
!pip install pdfplumber
clear_output()  

### Import libraries

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

from tqdm.notebook import tqdm
import pandas as pd
# from tabulate import tabulate
import pdfplumber
import glob
# import camelot
import os
import re
from functools import reduce
from dateutil import parser
%load_ext autotime

time: 428 µs (started: 2022-12-07 16:10:16 +00:00)


### Utils function

In [3]:
def get_trigrams(string):
    """
    Take a string and return a list of 3-grams.
    """
    s = string.strip().lower()
    return [s[i:i+3] for i in list(range(len(s) - 2))]


def string_similarity(str1, str2):
    """
    Perform bigram comparison between two strings
    and return a percentage match in decimal form.
    Input:
      str1: <str> key extracted from table
      str2: <str> keyword
    Return <float> score of similar between 2 strings
    """
    str1 = re.sub('[^A-Za-z0-9]+', '', str1.replace('\n', ''))
    str2 = re.sub('[^A-Za-z0-9]+', '', str2.replace('\n', ''))
    pairs1 = get_trigrams(str1)
    pairs2 = get_trigrams(str2)
    union = len(pairs1) + len(pairs2)
    hit_count = 0
    i = 0
    while i < len(pairs1):
      j = 0
      while j < len(pairs2):
        if pairs1[i] == pairs2[j]:
          pairs1.pop(i)
          pairs2.pop(j)
          hit_count += 1
          # s = True
          if i >= len(pairs1):break
        else:
          j += 1
      i += 1
        
    return (2.0 * hit_count) / union if union != 0 else 0

time: 3.74 ms (started: 2022-12-07 16:10:16 +00:00)


In [4]:
def contains_number(string):
  '''
  Check if string contain number or not
  '''
  return bool(re.search(r'\d', string)) or string.lower() == 'nil'

def get_key_value(tables, keys, threshold=0.8):
  """
  Extract needed field value based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is the value of the needed field and the second is the confident of mapping
  """
  max_score = 0
  result = None

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          max_score = max(scores)
          result = temp[j+1]

  if max_score > threshold:
    return result, max_score
  return None, None

def get_company_name(tables, keys, threshold=0.8):
  """
  Extract company name value based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is the value of company name and the second is the confident of mapping
  """
  results = []
  all_scores = []

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > threshold and j < len(temp)-1:
          if len(temp[j+1:]) == 2: # Apply heuristic in case which have an extra column between company key and company value in the file
            results += [temp[j+2]]
            all_scores += [ max(scores) ]
          else:
            results += [temp[j+1]]
            all_scores += [ max(scores) ]

  # The idea is find all values that have key matching score greater than threshold and return the first not N/A in company name value
  ids = list(filter(lambda x: 'n/a' not in results[x].lower(), range(len(results)))) # Apply heuristic to skip the N/A field value of company name
  results = [results[i] for i in ids]
  all_scores = [all_scores[i] for i in ids]
  if len(results) > 0:
    return results[0], all_scores[0]
  return None, None

def get_audit_recommendation(tables, rcm_keys, threshold=0.8):
  """
  Extract audit recommendation value based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is the audit recommendation table and the second is the confident of mapping
  """

  temp_key = 'Based on the result of this audit the audit team recommends that certification be' # Apply for case which have audit recommendation in 1 cell instead of have its own table
  rcm_tables = []
  for i in range(len(tables)):
    table = tables[i]
    for _row in range(len(table)):
      temp = np.array(list(table.iloc[_row]))
      temp = list(temp[temp != ''])
      if len(temp) == 0: continue
      
      scores = list(map(lambda x: string_similarity(temp[0], x), rcm_keys))
      if max(scores) > threshold:
        rcm_tables += [tables[i], tables[i+1]]
      
      if len(temp) == 1 and temp_key.lower() in temp[0].lower():
        return temp[0]
    
  results = []
  for i in range(0, len(rcm_tables), 2):
    rcm_table = rcm_tables[i:i+2]
    # Merge 2 continuous table if they have the same number of columns and the total number of rows not too large (in case table in 2 continuous page)
    if rcm_table[0].shape[1] == rcm_table[1].shape[1]:
      result_rcm_table = pd.concat(rcm_table, axis=0, ignore_index=True)
      if result_rcm_table.shape[0] > 4:
        result_rcm_table = rcm_table[0]
    else:
      result_rcm_table = rcm_table[0]
    
    results += [result_rcm_table]
  if len(results) == 0:
    return None
  return results[0].to_string() # Return the first matched table


def get_expiration_date_value(tables, keys, threshold=0.8):
  """
  Extract audit expiration date value based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is expiration date value and the second is the confident of mapping
  """
  max_score = 0
  result = None
  t = ''
  all_dates = []

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) >= max_score and j < len(temp)-1:
          max_score = max(scores)
          s = ' '.join(temp[j+1:]) if len(temp[j+1:]) < 3 else temp[j+1]
          try: # Some case parser get errors with parsing the date (no date in string)
            result = parser.parse(s, fuzzy=True)
            if max_score > threshold:
              all_dates += [ result ]
          except: pass

  if max_score > threshold and len(all_dates) > 0:
    return max(all_dates).strftime("%m/%d/%Y"), max_score
  return None, None

def get_minor_value(tables, keys, threshold=0.8):
  """
  Extract GMO minor value based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is minor number value and the second is the confident of mapping
  """

  max_score = 0
  result = None

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != '']) # Get all non-null column of i'th row
      if len(temp) > 2:
        continue
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          max_score = max(scores)
          result = temp[j+1]

  if max_score > threshold and contains_number(result): # The minor value should have number inside it
    return result, max_score
  return None, None


def get_audit_type_value(tables, keys, threshold=0.8):
  """
  Extract audit type based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is audit type value and the second is the confident of mapping
  """
  max_score = 0
  result = None

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != '']) # Get all non-null column of i'th row
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          if max(scores) > threshold and len(temp) - 1 - j > 1: # In case have more than 1 row in its right side
            if '\n' in ' '.join(temp[j+1:]): # If checkbox than skip (Skip the checkbox at this time)
              return None, None
            else:
              max_score = max(scores)
              result = '\n'.join(temp[j+1:])
          else:
            max_score = max(scores)
            result = temp[j+1]

  if max_score > threshold:
    return result, max_score
  return None, None


def get_gmo_value(tables, keys, threshold=0.8):
  """
  Extract audit GMO number (minor | critical) based on list all keys might have of that field
  Input:
    tables: <List[pd.DataFrame]> all tables extracted from pdf file
    keys: <list[str]> list contains all keys might have of the needed field
    threshold: <float> the threshold for mapping between needed keys and all keys of tables
  Return a tuple where the first is number gmo value and the second is the confident of mapping
  """

  max_score = 0
  result = None

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != '']) # Get all non-null column of i'th row
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          max_score = max(scores)
          result = temp[j+1]

  if max_score > threshold and contains_number(result):
    return result, max_score
  return None, None

time: 24.7 ms (started: 2022-12-07 16:10:16 +00:00)


In [5]:
from statistics import mode
from scipy import stats
from collections import Counter
from collections.abc import Iterable

def keep_visible_lines(mode_value):
    """
    If the object is a ``rect`` type, keep it only if the lines are visible.

    A visible line is the one having ``non_stroking_color`` as 0.
    """
    def process(obj):
      if obj['object_type'] == 'rect':
          v = tuple(obj['non_stroking_color']) if isinstance(obj['non_stroking_color'], Iterable) else obj['non_stroking_color']
          return v in mode_value
      return True
    return process

def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges
def foo(e):
  '''
  Filter to get all small peace of edge in pdf file
  '''
  dim = "height" if e["orientation"] == "v" else "width"
  return e[dim] <= 1

def get_tables_from_file(pdf_path):
  '''
  Read all tables from pdf path
  Input 
    pdf_path: <str> path to pdf file
  Return
    List of all tables, each is a pd.DataFrame object
  '''
  tables = []
  with pdfplumber.open(pdf_path) as pdf: 
    for page_id, page in enumerate(pdf.pages):

      # Get all small peace of edges in pdf file, then we can get the colors of border from that
      temp = list(filter(lambda x: foo(x), page.edges))
      non_strokings = list(map(lambda x: tuple(x['non_stroking_color']) if isinstance(x['non_stroking_color'], Iterable) else x['non_stroking_color'], temp))
      border_colors = list(dict(Counter(non_strokings)).keys())

      if len(page.edges) > 20:
        filter_fn = keep_visible_lines(border_colors)
        page = page.filter(filter_fn) # Filter all invisible line (which can affect the table detection)
      
      ts = {
        "vertical_strategy": "explicit",
        "horizontal_strategy": "explicit",
        "explicit_vertical_lines": curves_to_edges(page.curves) + page.edges,
        "explicit_horizontal_lines": curves_to_edges(page.curves) + page.edges,
      } if len(page.curves + page.edges) > 0 else {
        "vertical_strategy": "lines",
        "horizontal_strategy": "lines",
      }
      page_tables = page.find_tables(ts)
      tables += page_tables
    tables = list(map(lambda table: pd.DataFrame(table.extract()).fillna(''), tables))
  return tables
      

time: 776 ms (started: 2022-12-07 16:10:16 +00:00)


Define the keywords

In [6]:
# Define keyword for each field

company_name_keys = ['Company name', 'Registered legal name', 'Organisation']
address_keys = ['Address', 'Location', 'Street address, city, country', 'location/address', 'Address Line 1']
num_critical_keys = ['critical', 'critical non-conformities', 'Number of critical non-conformities identified during this audit', "Critical NCR’s"]
num_major_keys = ['major', 'major non-conformities', 'Number of major non-conformities identified during this audit', "Major NCR’s"]
num_minor_keys = ['minor', 'minor non-conformities', 'Number of minor non-conformities identified during this audit', "Minor NCR’s"]
audit_type_keys = ['Audit type']
prev_audit_type_keys = ['Previous Audit Type'] # Use in case some files use the same audit type as previous
rcm_keys = ['Audit Score/ Recommendation', 'Recommendation from this audit', 'Audit recommendation']
audit_company_keys = ['CB Name and office location', 'CB conducting audit', 'Certification Body', 'Accreditation Body(s)']
rcm_keys = ['Audit Score/ Recommendation', 'Recommendation from this audit', 'Audit recommendation']
rcm_one_row_keys = ['Based on the result of this audit the audit team recommends that certification be']
expiration_date_keys = ['Expiration Date', 'Expiry date', 'Certificate expiry date']

time: 2.05 ms (started: 2022-12-07 16:10:17 +00:00)


### Start inference

In [7]:
sheet_id = "1ixtNKdaaFW8XNROXMWWveuCxHPErju5UUgwDw5PjM_w"
sheet_name = "Files_and_Type"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)
all_subpaths = list(df['Full Path'])

all_paths = []
for path in all_subpaths:
  t = glob.glob(f'/content/drive/MyDrive/OCR NLP Stage 2/FSSC/Documents/**/{path}', recursive=True)
  all_paths += t

filenames = list(sorted(glob.glob('/content/drive/MyDrive/otraSorted_better/otraSorted/FSSC_Audit/*.pdf') ))


time: 7.41 s (started: 2022-12-07 16:10:17 +00:00)


Run 1 file

In [ ]:
fpath = filenames[15]
tables = get_tables_from_file(fpath)
company_name, cpn_score = get_company_name(tables, company_name_keys, threshold=0.8)
address, addr_score = get_key_value(tables, address_keys, threshold=0.8)
num_critical, critical_score = get_gmo_value(tables, num_critical_keys, threshold=0.8)
num_major, major_score = get_gmo_value(tables, num_major_keys, threshold=0.8)
num_minor, minor_score = get_minor_value(tables, num_minor_keys, threshold=0.8)

audit_type, audit_type_score = get_audit_type_value(tables, audit_type_keys, threshold=0.8)
if not audit_type:
  audit_type, audit_type_score = get_key_value(tables, prev_audit_type_keys, threshold=0.8)
audit_company, audit_company_score = get_key_value(tables, audit_company_keys, threshold=0.8)

audit_recommendation = get_audit_recommendation(tables, rcm_keys, threshold=0.8)
expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)


result = {
    'Company name': [company_name],
    'Address': [address],
    'Expiration date': [expiration_date],
    'Audit company name': [audit_company],
    'Audit type': [audit_type],
    'Number critical': [num_critical],
    'Number major': [num_major],
    'Number minor': [num_minor],
    'Recommendation': [audit_recommendation]
}

result = pd.DataFrame(index=list(result.keys()), data=list(result.values()), columns=['Result'])
print(result.to_markdown(tablefmt='grid'))

['Audit objective', 'The objectives of this audit were:  \nTo determine conformity of the management system, or parts \nof it with audit criteria and its:  \n•  ability to ensure applicable statutory, regulatory and \ncontractual requirements are met,  \n•  effectiveness to ensure the client can reasonably expect \nto achieve specified objectives, and  \n•  ability to identify as applicable areas for potential \nimprovement.']
['Audit criteria', 'ISO22000:2018, ISO/TS22002-1 and FSSC additional \nrequirements \nThe defined processes and documentation of the \nmanagement system developed by the organization  \nStatutory/regulatory requirements (as applicable) \nCustomer requirements (as applicable)']
['Audit type', 'Recertification', 'Announced']
+--------------------+-------------------------------------------------------+
|                    | Result                                                |
+====================+=======================================================+
| Compa

Run on all old dataset

In [8]:
def write_to_file(message, fname='./old_data_result2.txt'):
  with open(fname, 'a') as f:
    f.write(f'{message}\n')

_id = 0
for fpath in tqdm(all_paths):
  tables = get_tables_from_file(fpath)
  company_name, cpn_score = get_company_name(tables, company_name_keys, threshold=0.8)
  address, addr_score = get_key_value(tables, address_keys, threshold=0.8)
  num_critical, critical_score = get_gmo_value(tables, num_critical_keys, threshold=0.8)
  num_major, major_score = get_gmo_value(tables, num_major_keys, threshold=0.8)
  num_minor, minor_score = get_minor_value(tables, num_minor_keys, threshold=0.8)

  audit_type, audit_type_score = get_audit_type_value(tables, audit_type_keys, threshold=0.8)
  if not audit_type:
    audit_type, audit_type_score = get_key_value(tables, prev_audit_type_keys, threshold=0.8)
  audit_company, audit_company_score = get_key_value(tables, audit_company_keys, threshold=0.8)

  audit_recommendation = get_audit_recommendation(tables, rcm_keys, threshold=0.8)
  expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)


  result = {
      'Company name': [company_name],
      'Address': [address],
      'Expiration date': [expiration_date],
      'Audit company name': [audit_company],
      'Audit type': [audit_type],
      'Number critical': [num_critical],
      'Number major': [num_major],
      'Number minor': [num_minor],
      'Recommendation': [audit_recommendation]
  }

  result = pd.DataFrame(index=list(result.keys()), data=list(result.values()), columns=['Result'])
  write_to_file(f'===================================={_id+3}\n{os.path.basename(fpath)}\n\n')
  write_to_file(result.to_markdown(tablefmt='grid'))
  write_to_file('\n\n\n\n')
  _id += 1

  0%|          | 0/39 [00:00<?, ?it/s]

time: 7min 42s (started: 2022-12-07 16:10:24 +00:00)


In [ ]:
def write_to_file(message, fname='./production_data_result2.txt'):
  with open(fname, 'a') as f:
    f.write(f'{message}\n')

_id = 0
for fpath in tqdm(filenames):
  tables = get_tables_from_file(fpath)
  company_name, cpn_score = get_company_name(tables, company_name_keys, threshold=0.8)
  address, addr_score = get_key_value(tables, address_keys, threshold=0.8)
  num_critical, critical_score = get_gmo_value(tables, num_critical_keys, threshold=0.8)
  num_major, major_score = get_gmo_value(tables, num_major_keys, threshold=0.8)
  num_minor, minor_score = get_minor_value(tables, num_minor_keys, threshold=0.8)

  audit_type, audit_type_score = get_audit_type_value(tables, audit_type_keys, threshold=0.8)
  if not audit_type:
    audit_type, audit_type_score = get_key_value(tables, prev_audit_type_keys, threshold=0.8)
  audit_company, audit_company_score = get_key_value(tables, audit_company_keys, threshold=0.8)

  audit_recommendation = get_audit_recommendation(tables, rcm_keys, threshold=0.8)
  expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)


  result = {
      'Company name': [company_name],
      'Address': [address],
      'Expiration date': [expiration_date],
      'Audit company name': [audit_company],
      'Audit type': [audit_type],
      'Number critical': [num_critical],
      'Number major': [num_major],
      'Number minor': [num_minor],
      'Recommendation': [audit_recommendation]
  }
  result = pd.DataFrame(index=list(result.keys()), data=list(result.values()), columns=['Result'])
  write_to_file(f'===================================={_id+3}\n{os.path.basename(fpath)}\n\n')
  write_to_file(result.to_markdown(tablefmt='grid'))
  write_to_file('\n\n\n\n')
  _id += 1


  0%|          | 0/29 [00:00<?, ?it/s]

time: 5min 16s (started: 2022-12-07 05:39:10 +00:00)
